In [1]:
import os
os.environ['NUMPY_EXPERIMENTAL_ARRAY_FUNCTION'] = '0'

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import gcsfs # A pythonic file-system interface to Google Cloud Storage
from tqdm.autonotebook import tqdm

%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina' 

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  if __name__ == '__main__':


In [8]:
from scipy.interpolate import griddata

In [2]:
# Cluster was created via the dask labextension
# Delete this cell and replace with a new one

from dask.distributed import Client
from dask_kubernetes import KubeCluster

cluster = KubeCluster()
cluster.adapt(minimum=1, maximum=20, interval='2s')
client = Client(cluster)
client

distributed.scheduler - INFO - Clear task state
/srv/conda/envs/notebook/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)
distributed.scheduler - INFO -   Scheduler at:   tcp://10.32.19.15:42741
distributed.scheduler - INFO -   dashboard at:                    :45261
distributed.scheduler - INFO - Receive client connection: Client-afdbd440-3d2f-11ea-82e3-a6362a4112d1
distributed.core - INFO - Starting established connection


Client Scheduler: tcp://10.32.19.15:42741 Dashboard: /user/0000-0002-3606-2575/proxy/45261/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


distributed.scheduler - INFO - Register tcp://10.32.9.22:32799
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.9.22:32799
distributed.core - INFO - Starting established connection


In [9]:
def func_latlon_regrid(lat_n_regrid, lon_n_regrid, lat_min_regrid, lat_max_regrid, lon_min_regrid, lon_max_regrid): 
    Lat_regrid_1D=zeros ((lat_n_regrid));
    Lat_bound_regrid = zeros ((lat_n_regrid,2)); Lat_bound_regrid[0,0]=-90;  Lat_bound_regrid[0,1]=Lat_bound_regrid[0,0] + (180/lat_n_regrid); Lat_regrid_1D[0]=(Lat_bound_regrid[0,0]+Lat_bound_regrid[0,1])/2
    for ii in range(1,lat_n_regrid):
        Lat_bound_regrid[ii,0]=Lat_bound_regrid[ii-1,1]
        Lat_bound_regrid[ii,1]=Lat_bound_regrid[ii,0] +  (180/lat_n_regrid)
        Lat_regrid_1D[ii]=(Lat_bound_regrid[ii,0]+Lat_bound_regrid[ii,1])/2
    #### Longitude Bounds ####
    Lon_regrid_1D=zeros ((lon_n_regrid));
    Lon_bound_regrid = zeros ((lon_n_regrid,2)); Lon_bound_regrid[0,0]=0;  Lon_bound_regrid[0,1]=Lon_bound_regrid[0,0] + (360/lon_n_regrid); Lon_regrid_1D[0]=(Lon_bound_regrid[0,0]+Lon_bound_regrid[0,1])/2
    for ii in range(1,lon_n_regrid):
        Lon_bound_regrid[ii,0]=Lon_bound_regrid[ii-1,1]
        Lon_bound_regrid[ii,1]=Lon_bound_regrid[ii,0] +  (360/lon_n_regrid)
        Lon_regrid_1D[ii]=(Lon_bound_regrid[ii,0]+Lon_bound_regrid[ii,1])/2
    
    return Lat_regrid_1D, Lon_regrid_1D, Lat_bound_regrid, Lon_bound_regrid



In [3]:
df = pd.read_csv('https://storage.googleapis.com/pangeo-cmip6/pangeo-cmip6-zarr-consolidated-stores.csv')
df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
0,AerChemMIP,BCC,BCC-ESM1,piClim-CH4,r1i1p1f1,Amon,ch4,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/piClim-CH4/...,NaN
1,AerChemMIP,BCC,BCC-ESM1,piClim-CH4,r1i1p1f1,Amon,clt,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/piClim-CH4/...,NaN
2,AerChemMIP,BCC,BCC-ESM1,piClim-CH4,r1i1p1f1,Amon,co2,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/piClim-CH4/...,NaN
3,AerChemMIP,BCC,BCC-ESM1,piClim-CH4,r1i1p1f1,Amon,evspsbl,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/piClim-CH4/...,NaN
4,AerChemMIP,BCC,BCC-ESM1,piClim-CH4,r1i1p1f1,Amon,hfls,gn,gs://cmip6/AerChemMIP/BCC/BCC-ESM1/piClim-CH4/...,NaN


In [4]:
df['table_id'].unique()

array(['Amon', 'Omon', 'day', 'AERmon', 'AERmonZ', 'CFmon', 'fx', 'Lmon',
       'SImon', 'Oyr', 'LImon', 'Ofx', '3hr', '6hrLev', 'Oday', 'Aclim',
       'Emon', 'CFday', '6hrPlev', 'CF3hr', 'Oclim', 'SIclim', 'Eclim',
       'Odec', 'AERhr', 'IfxGre', 'ImonGre', '6hrPlevPt', 'E3hr'],
      dtype=object)

In [5]:
df['activity_id'].unique()

array(['AerChemMIP', 'C4MIP', 'CFMIP', 'CMIP', 'DAMIP', 'DCPP', 'FAFMIP',
       'GMMIP', 'HighResMIP', 'LS3MIP', 'LUMIP', 'OMIP', 'PMIP', 'RFMIP',
       'ScenarioMIP'], dtype=object)

In [6]:
df['experiment_id'].unique()

array(['piClim-CH4', 'piClim-NTCF', 'piClim-control', 'ssp370',
       'piClim-2xDMS', 'piClim-2xdust', 'piClim-2xfire', 'piClim-2xss',
       'piClim-BC', 'piClim-HC', 'piClim-N2O', 'piClim-OC', 'piClim-SO2',
       '1pctCO2-bgc', 'esm-ssp585', 'hist-bgc', 'amip-4xCO2',
       'amip-future4K', 'amip-m4K', 'amip-p4K', 'amip', 'abrupt-2xCO2',
       'abrupt-solp4p', 'abrupt-0p5xCO2', 'amip-lwoff', 'amip-p4K-lwoff',
       'abrupt-solm4p', 'aqua-4xCO2', 'aqua-control-lwoff',
       'aqua-control', 'aqua-p4K-lwoff', 'aqua-p4K', '1pctCO2',
       'abrupt-4xCO2', 'historical', 'piControl', 'esm-hist',
       'esm-piControl', 'ssp126', 'ssp245', 'ssp585',
       'esm-piControl-spinup', 'piControl-spinup', 'hist-GHG', 'hist-aer',
       'hist-nat', 'hist-CO2', 'hist-sol', 'hist-stratO3', 'hist-volc',
       'ssp245-GHG', 'ssp245-aer', 'ssp245-nat', 'ssp245-stratO3',
       'dcppA-hindcast', 'dcppA-assim', 'dcppC-hindcast-noAgung',
       'dcppC-hindcast-noElChichon', 'dcppC-hindcast-noPinatub

In [7]:
df['variable_id'].unique()

array(['ch4', 'clt', 'co2', 'evspsbl', 'hfls', 'hfss', 'hus', 'o3', 'pr',
       'prsn', 'prw', 'ps', 'psl', 'rlds', 'rlus', 'rlut', 'rlutcs',
       'rsds', 'rsdt', 'rsus', 'rsut', 'rsutcs', 'ta', 'tas', 'tauu',
       'tauv', 'ts', 'ua', 'uas', 'va', 'vas', 'zg', 'tasmax', 'tasmin',
       'zos', 'airmass', 'cdnc', 'co', 'emidust', 'emiss', 'mmrbc',
       'mmrdust', 'mmroa', 'mmrso4', 'mmrss', 'no', 'no2', 'o3loss',
       'o3prod', 'od550aer', 'oh', 'ptp', 'rlutaf', 'rlutcsaf', 'rsutaf',
       'rsutcsaf', 'so2', 'toz', 'wa', 'hurs', 'huss', 'areacella',
       'clisccp', 'cLeaf', 'cVeg', 'gpp', 'lai', 'npp', 'ra', 'tran',
       'sisnthick', 'chl', 'detoc', 'diftrblo', 'difvho', 'difvso',
       'dissic', 'dissicabio', 'dissicnat', 'fgco2', 'fgco2abio',
       'fgco2nat', 'no3', 'o2', 'phyc', 'phyn', 'pon', 'talk', 'zooc',
       'nbp', 'siconc', 'sisnconc', 'fgo2', 'hfds', 'sos', 'tos', 'calc',
       'dfe', 'dissoc', 'expc', 'expn', 'expp', 'expsi', 'graz', 'nh4',
       'ph', '

In [18]:
df_pr_mon = df[(df.table_id == 'Amon') & (df.variable_id == 'pr') & (df.activity_id == 'DCPP') & (df.experiment_id == 'dcppA-hindcast')]
#df_pr_mon = df_pr_mon[ df_pr_mon['member_id'] == 'r1i1p1f1']
#df_pr_mon = df_pr_mon[ df_pr_mon['grid_label'] == 'gn']
#df_pr_mon = df_pr_mon[ df_pr_mon['member_id'].apply(lambda run: run[2]) == 'i' ] # Removing runs that have double digit run numbers like r10i1p1f2
#df_pr_mon = df_pr_mon[ df_pr_mon['member_id'].apply(lambda run: run[1]) == '1' ] # Only select 1st runs (for example r1i1p1f1)
len(df_pr_mon)

1982

In [19]:
run_counts = df_pr_mon.groupby(['source_id', 'experiment_id'])['zstore'].count()
run_counts

source_id  experiment_id 
CanESM5    dcppA-hindcast    1140
MIROC6     dcppA-hindcast     590
NorCPM1    dcppA-hindcast     252
Name: zstore, dtype: int64

In [23]:
df_pr_mon[df_pr_mon.source_id=='NorCPM1'].head(10)

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year
188737,DCPP,NCC,NorCPM1,dcppA-hindcast,r10i2p1f1,Amon,pr,gn,gs://cmip6/DCPP/NCC/NorCPM1/dcppA-hindcast/s19...,1960.0
188749,DCPP,NCC,NorCPM1,dcppA-hindcast,r1i1p1f1,Amon,pr,gn,gs://cmip6/DCPP/NCC/NorCPM1/dcppA-hindcast/s19...,1960.0
188762,DCPP,NCC,NorCPM1,dcppA-hindcast,r1i2p1f1,Amon,pr,gn,gs://cmip6/DCPP/NCC/NorCPM1/dcppA-hindcast/s19...,1960.0
188786,DCPP,NCC,NorCPM1,dcppA-hindcast,r3i1p1f1,Amon,pr,gn,gs://cmip6/DCPP/NCC/NorCPM1/dcppA-hindcast/s19...,1960.0
188809,DCPP,NCC,NorCPM1,dcppA-hindcast,r4i2p1f1,Amon,pr,gn,gs://cmip6/DCPP/NCC/NorCPM1/dcppA-hindcast/s19...,1960.0
188841,DCPP,NCC,NorCPM1,dcppA-hindcast,r6i1p1f1,Amon,pr,gn,gs://cmip6/DCPP/NCC/NorCPM1/dcppA-hindcast/s19...,1960.0
188849,DCPP,NCC,NorCPM1,dcppA-hindcast,r6i2p1f1,Amon,pr,gn,gs://cmip6/DCPP/NCC/NorCPM1/dcppA-hindcast/s19...,1960.0
188870,DCPP,NCC,NorCPM1,dcppA-hindcast,r8i1p1f1,Amon,pr,gn,gs://cmip6/DCPP/NCC/NorCPM1/dcppA-hindcast/s19...,1960.0
188900,DCPP,NCC,NorCPM1,dcppA-hindcast,r10i1p1f1,Amon,pr,gn,gs://cmip6/DCPP/NCC/NorCPM1/dcppA-hindcast/s19...,1961.0
189005,DCPP,NCC,NorCPM1,dcppA-hindcast,r6i2p1f1,Amon,pr,gn,gs://cmip6/DCPP/NCC/NorCPM1/dcppA-hindcast/s19...,1961.0


distributed.scheduler - INFO - Remove worker tcp://10.32.9.22:32799
distributed.core - INFO - Removing comms to tcp://10.32.9.22:32799
distributed.scheduler - INFO - Lost all workers
distributed.scheduler - INFO - Register tcp://10.32.9.3:38357
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.32.9.3:38357
distributed.core - INFO - Starting established connection
